# 🌦 IoT Rain Data Examples — Citywise & Devicewise Analysis

This notebook demonstrates how to fetch, process IoT rainfall data using the `fetch_iot()` API and helper functions.  
Each example focuses on a specific use case, showing how to handle pagination, aggregation (MAX).

---

### ✅ **Examples Covered**

1. **Example 1:** Single device rain chart 
2. **Example 2:** Paginated Top 10 devices by MAX rain (Bochum, Oct 1–7, 2025)  
4. **Example 3:** Lübeck city — MAX rain per device (Sep 14–Oct 02, 2025)

---

### ⚙️ **Setup**

Before running the examples:
1. Ensure `fetch_iot()` and `to_df()` are defined.  
2. Verify that your `.env` and API credentials are loaded correctly.  
3. Run the setup and import cells first.


In [43]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated"
)


In [44]:
# Setup & imports
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

# allow imports when running from /notebooks
sys.path.append("..")
sys.path.append(".")

# 1) load API client
from utils.iot_client import fetch_iot

# 2) load .env if present (kept for convenience)
from dotenv import load_dotenv
load_dotenv(os.path.join("..", "config", ".env")) or load_dotenv(os.path.join("config", ".env"))

print("Ready")

Ready


In [45]:
def to_df(rows):
    """
    Convert list[dict] -> DataFrame with light parsing.
    - parse minute_ts as UTC datetime
    - coerce rain_value to numeric
    - pre-sort by time if present
    """
    if not rows:
        return pd.DataFrame()
    df = pd.DataFrame(rows)
    if "minute_ts" in df.columns:
        df["minute_ts"] = pd.to_datetime(df["minute_ts"], utc=True, errors="coerce")
    if "rain_value" in df.columns:
        df["rain_value"] = pd.to_numeric(df["rain_value"], errors="coerce")
    if {"minute_ts", "rain_value"}.issubset(df.columns):
        df = df.sort_values("minute_ts")
    return df


def paginate_iot(*, city=None, from_ts=None, to_ts=None,
                 only_with_known_location=True, source="auto",
                 order="asc", page=1000, max_rows=50_000, verbose=True):
    """
    Pulls multiple pages from /iot using limit/offset until no more rows (or cap).
    Returns a single DataFrame.
    """
    all_rows, offset = [], 0
    while True:
        batch = fetch_iot(
            city=city,
            from_ts=from_ts, to_ts=to_ts,
            only_with_known_location=only_with_known_location,
            source=source,
            order=order,
            limit=page, offset=offset
        )
        if not batch:
            break
        all_rows.extend(batch)
        offset += page
        if verbose:
            print(f"Fetched {len(all_rows):,} rows...")
        if len(all_rows) >= max_rows:
            if verbose:
                print(f"Hit safety cap: {max_rows:,} rows.")
            break
    return to_df(all_rows)

# Example 1 — Single-Device View (raw data)

### Purpose
Provide a data preview for a single IoT rain sensor.

### What this example does
1. Fetch one time window of IoT rain data for **Bochum**  
2. Convert the API response into a pandas DataFrame  
3. Clean timestamp and rain columns  
4. Automatically pick **one device** (the device with the most rows)  
5. Display the first rows for that device as a plain table  

### Output
A DataFrame showing:
- `minute_ts` – timestamp  
- `dev_eui` – device ID  
- `rain_value` – rain measurement



### Fetch one batch of Bochum data (no pagination)

In [46]:
# Fetch one batch of Bochum data (no pagination)

import pandas as pd

# Fetch one window of data for Bochum
rows1 = fetch_iot(
    city="Bochum",
    from_ts="2025-10-01T00:00:00Z",
    to_ts="2025-10-07T23:59:59Z",
    only_with_known_location=True,
    source="auto",
    order="asc",
    limit=1000,   # single request, no paging
    offset=0,
)

# Convert API rows -> DataFrame and keep it in 'df'
df = to_df(rows1)

print(f"Rows returned: {len(df)}")
display(df.head())



Rows returned: 1000


,dev_eui,minute_ts,rain_value,quality_flag,status,dev_name,longitude,latitude,city,sensor_site_name
0,0080E115004E327C,2025-10-01 00:00:00+00:00,NaN,9,NaN,None,7.213700,51.440100,Bochum,0080E115004E327C_KemnaderStraße_WeitmarMark
28,0080E115004E7C64,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.241546,51.469333,Bochum,0080E115004E7C64_SportplatzPappelbusch_Altenbo...
27,0080E115004E6993,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.152601,51.479614,Bochum,0080E115004E6993_LudwigSteilHaus_Wattenscheid
26,0080E115004E611F,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.268676,51.491582,Bochum,0080E115004E611F_Kornharpen_KornhapenerStraße
25,0080E115004E6106,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.142429,51.500400,Bochum,0080E115004E4707_Osterfeldstraße_FFGünnigfeld


### Shows a table for one device (most active sensor in this window).


In [47]:
# Example 1 – Single device table

import pandas as pd

# Make sure the fetch cell above has been run
assert "df" in globals(), "Run the fetch cell above first so that 'df' exists."

# Basic cleanup
d = df.copy()
d["minute_ts"]  = pd.to_datetime(d["minute_ts"], errors="coerce", utc=True)
d["rain_value"] = pd.to_numeric(d["rain_value"], errors="coerce")
d = d.dropna(subset=["minute_ts"])

# Pick one device (the one with most rows)
device_id = d["dev_eui"].value_counts().idxmax()

device_df = (
    d[d["dev_eui"] == device_id]
      .sort_values("minute_ts")
      [["minute_ts", "dev_eui", "rain_value"]]
)

print(f"Device used: {device_id}")
print(f"Total rows for this device: {len(device_df)}")
display(device_df.head(20))


Device used: 0080E115004E327C
Total rows for this device: 35


,minute_ts,dev_eui,rain_value
0,2025-10-01 00:00:00+00:00,0080E115004E327C,NaN
29,2025-10-01 00:01:00+00:00,0080E115004E327C,NaN
58,2025-10-01 00:02:00+00:00,0080E115004E327C,NaN
88,2025-10-01 00:03:00+00:00,0080E115004E327C,NaN
116,2025-10-01 00:04:00+00:00,0080E115004E327C,NaN
145,2025-10-01 00:05:00+00:00,0080E115004E327C,NaN
174,2025-10-01 00:06:00+00:00,0080E115004E327C,NaN
203,2025-10-01 00:07:00+00:00,0080E115004E327C,NaN
233,2025-10-01 00:08:00+00:00,0080E115004E327C,NaN
261,2025-10-01 00:09:00+00:00,0080E115004E327C,NaN


## Example 2 — Bochum (Oct 1–7) with pagination

**Purpose**
fetch a longer time window of IoT rain data for **Bochum** using
`limit` and `offset` (pagination), and then see which devices recorded the
highest rain values.

**What this example does**

1. Fetches Bochum rain data for **2025-10-01 to 2025-10-07** using pagination.  
2. Combines all pages into a single pandas DataFrame.  
3. Shows a small preview of the paginated data.  
4. Calculates the **Top 10 devices** by maximum `rain_value` in that window.

**Outputs**

- A DataFrame **preview** (`df_paginated.head()`) for Bochum (Oct 1–7).  
- A second table: **Top 10 devices by MAX rain** for the same period.



In [48]:
## Example 2 – Fetch Bochum data with pagination

import pandas as pd

# Time window & city
from_ts = "2025-10-01T00:00:00Z"
to_ts   = "2025-10-07T23:59:59Z"
CITY    = "Bochum"

# Simple pagination settings
PAGE = 500      # rows per request
CAP  = 20_000   # safety cap on total rows


def paginate_iot(city, from_ts, to_ts,
                 only_with_known_location=True,
                 source="auto", order="asc",
                 page=PAGE, max_rows=CAP):
    """
    Fetch IoT data in pages and return a single DataFrame.
    Simple, partner-friendly version.
    """
    assert "fetch_iot" in globals() and "to_df" in globals(), \
        "Run the setup cell first (fetch_iot / to_df must be defined)."

    frames = []
    offset = 0
    total  = 0

    while total < max_rows:
        rows = fetch_iot(
            city=city,
            from_ts=from_ts,
            to_ts=to_ts,
            only_with_known_location=only_with_known_location,
            source=source,
            order=order,
            limit=page,
            offset=offset,
        )

        df_page = to_df(rows)
        if df_page is None or df_page.empty:
            break

        frames.append(df_page)
        total += len(df_page)
        offset += page

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

df_paginated = paginate_iot(
    city=CITY,
    from_ts=from_ts,
    to_ts=to_ts,
    only_with_known_location=True,
    source="auto",
    order="asc",
)

print(f"Rows in window: {len(df_paginated)}")
display(df_paginated.head())   


Rows in window: 20000


,dev_eui,minute_ts,rain_value,quality_flag,status,dev_name,longitude,latitude,city,sensor_site_name
0,0080E115004E327C,2025-10-01 00:00:00+00:00,NaN,9,NaN,None,7.213700,51.440100,Bochum,0080E115004E327C_KemnaderStraße_WeitmarMark
1,0080E115004E7C64,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.241546,51.469333,Bochum,0080E115004E7C64_SportplatzPappelbusch_Altenbo...
2,0080E115004E6993,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.152601,51.479614,Bochum,0080E115004E6993_LudwigSteilHaus_Wattenscheid
3,0080E115004E611F,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.268676,51.491582,Bochum,0080E115004E611F_Kornharpen_KornhapenerStraße
4,0080E115004E6106,2025-10-01 00:00:00+00:00,0.0,1,NaN,None,7.142429,51.500400,Bochum,0080E115004E4707_Osterfeldstraße_FFGünnigfeld


### Shows the Top 10 devices by maximum rain


In [49]:
## Example 2 – Top 10 devices by MAX rain 
if not df_paginated.empty and "rain_value" in df_paginated.columns:
    d = df_paginated.copy()
    d["rain_value"] = pd.to_numeric(d["rain_value"], errors="coerce")

    top_devs = (
        d.groupby("dev_eui", as_index=False)["rain_value"]
         .max()  # MAX over the window
         .rename(columns={"rain_value": "max_rain"})
         .sort_values("max_rain", ascending=False)
         .head(10)
    )

    print("Top 10 devices by MAX rain:")
    display(top_devs)
else:
    print("No data for this window or no rain_value column.")


Top 10 devices by MAX rain:


,dev_eui,max_rain
15,0080E115004E53D3,2.0
3,0080E115004E4686,2.0
27,0080E115004E6993,2.0
18,0080E115004E5716,2.0
1,0080E115004E436D,0.0
26,0080E115004E611F,0.0
25,0080E115004E6106,0.0
24,0080E115004E60D1,0.0
23,0080E115004E606E,0.0
22,0080E115004E5BCB,0.0


### Example 3 — Lübeck | MAX Rain per Device (Sep 14–Oct 02, 2025)

**Goal:** Fetch Lübeck data for the selected period and show a simple table of devices with the highest recorded rainfall.

**Steps:**
1. Download Lübeck data between **2025-09-14** and **2025-10-02** using safe pagination.  
2. Combine all pages into one DataFrame.  
3. Compute **MAX rainfall per device**.  
4. Display the **Top 10 devices** ranked by peak rainfall.

**👉 Output:** A clean DataFrame (top 10 devices by max rainfall).


In [50]:

# ---- Lübeck | MAX rain per device (Sep 14–Oct 02, 2025) — MINIMAL ----
import pandas as pd

city    = "Lübeck"
from_ts = "2025-09-14T00:00:00Z"
to_ts   = "2025-10-02T00:00:00Z"

PAGE, CAP, MAX_PAGES = 200, 4000, 8

def paginate_city(city, from_ts, to_ts, page=PAGE, cap=CAP, max_pages=MAX_PAGES):
    """Small, safe paginator. Returns a single DataFrame."""
    frames, offset, total, last_sig = [], 0, 0, None
    for _ in range(max_pages):
        rows = fetch_iot(
            city=city, from_ts=from_ts, to_ts=to_ts,
            only_with_known_location=True, source="auto", order="asc",
            limit=page, offset=offset,
        )
        d = to_df(rows)
        if d is None or d.empty:
            break
        sig = (tuple(d.iloc[0].values), tuple(d.iloc[-1].values))
        if sig == last_sig:         # repeated page protection
            break
        last_sig = sig

        frames.append(d)
        total += len(d)
        if len(d) < page or total >= cap:
            break
        offset += page

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

df = paginate_city(city, from_ts, to_ts)
if df.empty or "rain_value" not in df.columns:
    print("No data.")
else:
    df["rain_value"] = pd.to_numeric(df["rain_value"], errors="coerce")
    top = (
        df.groupby("dev_eui", as_index=False)["rain_value"]
          .max()
          .rename(columns={"rain_value": "max_rain"})
          .sort_values("max_rain", ascending=False)
    )
    display(top.head(10))


,dev_eui,max_rain
1,0080E115004E45ED,2.0
8,0080E115004E6120,2.0
9,0080E115004E7217,2.0
0,0080E115004E223A,1.0
2,0080E115004E4DB5,1.0
4,0080E115004E557C,1.0
5,0080E115004E5744,1.0
7,0080E115004E6040,1.0
10,0080E115004E72F2,1.0
11,0080E115004E761F,1.0
